In [35]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Загрузка данных

In [36]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [37]:
data = pd.read_excel('data_ford_price.xlsx') 

#  Отбор признаков: мотивация

## Предобработка данных

In [38]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

## Обучение модели

In [39]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4682.957


## Удаление избыточного признака

In [40]:
x.drop('lat', axis = 1, inplace = True)

In [41]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [42]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4672.930


#  Отбор признаков: классификация методов

## Метод рекурсивного исключения признаков

In [43]:
from sklearn.feature_selection import RFE

In [44]:
y = data['price']
x = data.drop(columns='price')

In [45]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [46]:
#Создаем модель
LR_model_rfe = LinearRegression()
#Создаем выборщика признаков
selector = RFE(LR_model_rfe, n_features_to_select=3, step=1)
#Обучаем его
selector = selector.fit(X_train, y_train)
#Выводим полученные названия полученных признаков
features_rfe = selector.get_feature_names_out()
features_rfe

array(['year', 'cylinders', 'lat'], dtype=object)

In [47]:
#Выделяем в отдельный датасет признаки выбранные по RFE способу.
X_train_rfe = X_train[features_rfe]
#Обучаем модель
LR_model_rfe.fit(X_train_rfe, y_train)
#Делаем предсказание
y_rfe_pred = LR_model_rfe.predict(X_test[features_rfe])
#Выводим значение средней абсолютной ошибки.
mae_rfe = mean_absolute_error(y_test, y_rfe_pred)
print('MAE on RFE features: %.3f' % mae_rfe)

MAE: 5096.570


##  МЕТОДЫ ВЫБОРА ПРИЗНАКОВ НА ОСНОВЕ ФИЛЬТРОВ

In [48]:
from sklearn.feature_selection import SelectKBest, f_regression

In [49]:
#Создаем выборщика признаков
selector = SelectKBest(f_regression, k=3)
#Обучаем его
selector.fit(X_train, y_train)
#Выводим полученные названия полученных признаков
features_kbest = selector.get_feature_names_out()
features_kbest

array(['year', 'cylinders', 'odometer'], dtype=object)

In [50]:
#Создаем модель
LR_model_kbest = LinearRegression()

In [51]:
#Выделяем в отдельный датасет признаки выбранные по SelectKBest способу.
X_train_kbest = X_train[features_kbest]
#Обучаем модель
LR_model_kbest.fit(X_train_kbest, y_train)
#Делаем предсказание
y_kbest_pred = LR_model_kbest.predict(X_test[features_kbest])
#Выводим значение средней абсолютной ошибки.
mae_kbest = mean_absolute_error(y_test, y_kbest_pred)
print('MAE on SelectKBest features: %.3f' % mae_kbest)

MAE: 4708.946


Вывод: лучшее значение средней абсолютной ошибки получено на полных данных. Если выбирать из значений средней абсолютной ошибки полученных на признаках отсеянных с помощью методов RFE и SelectKBest, то лучший результат за SelectKBest способом.